In [1]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
from EC_data_processing_lib import get_header_length
from EC_data_processing_lib import analyse_file
from EC_data_processing_lib import get_decimal_separator
from pandas import DataFrame as df

fig = go.Figure()

#the following section uploads the data. You need to update the dataset array when you add or remove files. 
CoNx_dict = {
'Ru' : 0.85,  #measured resistance in ohms
'data' : 'Data/2022_01_05 CoNx Ti electrode1 exp2_03_CP_C05.mpt', #file path the slash needs to be '/' rather than the '\' which is automatic
'label' : 'CoN<sub>X</sub>',
'cal' : 0.309, #calibration of the Hg/HgO electrode vs RHE converting to overpotential vs OER MUST BE UPDATED WHEN USING A DIFFERENT REF ELECTRODE
}

NiNx_dict = {
'Ru' : 0.85,  #measured resistance in ohms
'data' : 'Data/2022_01_04 NiNx Ti electrode1 CV post tafel, bulk_03_CP_C05.mpt',
'label' : 'NiN<sub>X</sub>',
'cal' : 0.309,
}

FeNx_dict = {
'Ru' : 0.85,  #measured resistance in ohms
'data' : 'Data/2022_01_11 FeNx Ti electrode set2_03_CP_C05.mpt',
'label' : 'FeN<sub>X</sub>',
'cal' : 0.309,
}

dataset = [CoNx_dict, NiNx_dict, FeNx_dict]

for i in dataset:
    headerlength = get_header_length(i['data'])
    data = pd.read_csv(i['data'], encoding='ansi', sep="\t", decimal=get_decimal_separator(i['data']),  skiprows=range(headerlength))
    voltage = data['<Ewe>/V']
    time = data['time/s']/60  #converts seconds to minutes
    time = time - time[0] #sets intitial time to 0
    endtime = time[-10:]
    voltage_corrected = data['<Ewe>/V'] - data['I/mA'] * i['Ru']/1000 - i['cal'] #does the iR correction converts to overpotential
    voltage_corrected = voltage_corrected*1000
    
    j = 0
    Final_Values = []
    # calculates the final overpotential values and prints them along with standard deviation    
    for j in (time[time>115]):
        final_η = df.mean(voltage_corrected)
        final_η = np.round(final_η)
        final_η_error = df.std(voltage_corrected)
        final_η_error = np.round(final_η_error)
        Final_Values = ([i['label'], float(final_η), float(final_η_error)]) 
    Final_Values = np.array(Final_Values)
    
    print(Final_Values)

    fig.add_trace(go.Scatter(
        x=time,
        y=voltage_corrected,
        name= i['label']
    )
    
    
    
    )
fig.update_xaxes(range=[0, 120],
    nticks=10,
    ticks='outside',
    showgrid=False, gridwidth=1, gridcolor='grey',
    zeroline=False, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')

fig.update_yaxes(range=[300, 2000],
    nticks=10,
    showgrid=False, gridwidth=1, gridcolor='grey',
    ticks='outside',
    zeroline=True, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')
fig.update_layout(
    #title="CV",
    xaxis_title="Time (min)",
    yaxis_title="Overpotential, η(mV) - iR<sub>u</sub>",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
        color="dimgrey"
        
    )
)

fig.update_layout({
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    'paper_bgcolor': 'rgba(0, 0, 0, 0)', #sets the background to be transparent... might appear black in .jpeg but will be transparent in the .svg
})
fig.update_layout(
    width = 500,
    height = 500)
fig.show()
fig.write_image('Chronopotentiometry.jpeg')
fig.write_image('Chronopotentiometry.svg')


['CoN<sub>X</sub>' '506.0' '7.0']
['NiN<sub>X</sub>' '938.0' '4.0']
['FeN<sub>X</sub>' '1539.0' '224.0']
